In [1]:
from uncertainties import unumpy as unp
import numpy as np

from pisa.core.distribution_maker import DistributionMaker
from pisa.analysis.analysis import Analysis
from pisa import ureg

<< PISA is running in double precision (FP64) mode; numba is running on CPU (single core) >>


In [2]:
test = DistributionMaker('resources/settings/pipeline/fast_example.cfg')
test_var = DistributionMaker('resources/settings/pipeline/varbin_example.cfg')

[ WARNING] /data/user/jweldert/miniconda3/envs/pisa/lib/python3.9/site-packages/pint/quantity.py:221: BehaviorChangeWarning: The way Pint handles NumPy operations has changed with the
implementation of NEP 18. Unimplemented NumPy operations will now fail instead of making
assumptions about units. Some functions, eg concat, will now return Quanties with units, where
they returned ndarrays previously. See https://github.com/hgrecco/pint/pull/905.

To hide this warning, wrap your first creation of an array Quantity with
warnings.catch_warnings(), like the following:

import numpy as np
import warnings
from pint import Quantity

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    Quantity([])

To disable the new behavior, see
https://www.numpy.org/neps/nep-0018-array-function-protocol.html#implementation

  warnings.warn(array_function_change_msg, BehaviorChangeWarning)

[ WARNING] renaming the outdated "oppo" flux keys in "nue_cc", in the future do not use those anymor

In [8]:
%%timeit
out = test.get_outputs(return_sum=True)

15.7 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit
out_var = test_var.get_outputs(return_sum=True)

33.7 ms ± 144 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
a = unp.nominal_values(out['total'].hist[:,:,0])
b = unp.nominal_values(out_var[0]['total'].hist)
np.all(np.isclose(a, b))

True

In [5]:
a = unp.std_devs(out['total'].hist[:,:,0])
b = unp.std_devs(out_var[0]['total'].hist)
np.all(np.isclose(a, b))

True

In [6]:
a = unp.nominal_values(out['total'].hist)[:,:,1]
b = unp.nominal_values(out_var[1]['total'].hist[:,1::2]+out_var[1]['total'].hist[:,0::2])
np.all(np.isclose(a, b))

True

In [7]:
a = unp.std_devs(out['total'].hist)[:,:,1]
b = unp.std_devs(out_var[1]['total'].hist[:,1::2]+out_var[1]['total'].hist[:,0::2])
np.all(np.isclose(a, b))

True

In [10]:
ana = Analysis()

In [11]:
# local minimizer
local_fit_minuit = {
    "method": "iminuit",
    "method_kwargs": {
        "errors": 0.1,
        "precision": 1e-14,  # default: double precision
        "tol": 1e-2,  # default: 0.1
        "run_simplex": False,
        "run_migrad": True
    },
    "local_fit_kwargs": None
}

# octant fit for local minimizer
fit_octant = {
    "method": "octants",
    "method_kwargs": {
        "angle": "theta23",
        "inflection_point": 45 * ureg.degrees,
    },
    "local_fit_kwargs": local_fit_minuit
}

In [16]:
test.params['aeff_scale'] = 0.9
test_var.params['aeff_scale'] = 0.9

In [19]:
%%time
result = ana.fit_recursively(
    data_dist=out,
    hypo_maker=test,
    metric="mod_chi2",
    external_priors_penalty=None,
    **fit_octant
)

                                                 (deg)                 
 iter   funcalls    mod_chi2   | delta_index    theta23     aeff_scale 
------ ---------- ------------ + ------------ ------------ ------------
  0        0       4.84685e+00 |  0.00000e+00  4.23000e+01  9.00000e-01
  0        1       4.86333e+00 |  1.00679e-03  4.23000e+01  9.00000e-01
  0        2       4.83135e+00 | -1.00679e-03  4.23000e+01  9.00000e-01
  0        3       4.84845e+00 |  1.00679e-04  4.23000e+01  9.00000e-01
  0        4       4.84525e+00 | -1.00679e-04  4.23000e+01  9.00000e-01
  0        5       4.85210e+00 |  0.00000e+00  4.23436e+01  9.00000e-01
  0        6       4.84234e+00 |  0.00000e+00  4.22560e+01  9.00000e-01
  0        7       4.84740e+00 |  0.00000e+00  4.23049e+01  9.00000e-01
  0        8       4.84631e+00 |  0.00000e+00  4.22951e+01  9.00000e-01
  0        9       4.52603e+00 |  0.00000e+00  4.23000e+01  9.03041e-01
  0        10      5.18060e+00 |  0.00000e+00  4.23000e+01  8.96

In [20]:
%%time
result_var = ana.fit_recursively(
    data_dist=out_var,
    hypo_maker=test_var,
    metric="mod_chi2",
    external_priors_penalty=None,
    **fit_octant
)

                                                 (deg)                 
 iter   funcalls    mod_chi2   | delta_index    theta23     aeff_scale 
------ ---------- ------------ + ------------ ------------ ------------
  0        0       5.13950e+00 |  0.00000e+00  4.23000e+01  9.00000e-01
  0        1       5.15597e+00 |  1.00679e-03  4.23000e+01  9.00000e-01
  0        2       5.12407e+00 | -1.00679e-03  4.23000e+01  9.00000e-01
  0        3       5.14110e+00 |  1.00679e-04  4.23000e+01  9.00000e-01
  0        4       5.13791e+00 | -1.00679e-04  4.23000e+01  9.00000e-01
  0        5       5.15366e+00 |  0.00000e+00  4.23436e+01  9.00000e-01
  0        6       5.12773e+00 |  0.00000e+00  4.22560e+01  9.00000e-01
  0        7       5.14081e+00 |  0.00000e+00  4.23044e+01  9.00000e-01
  0        8       5.13822e+00 |  0.00000e+00  4.22956e+01  9.00000e-01
  0        9       4.79933e+00 |  0.00000e+00  4.23000e+01  9.03041e-01
  0        10      5.49340e+00 |  0.00000e+00  4.23000e+01  8.96



CPU times: user 12.2 s, sys: 42.9 ms, total: 12.3 s
Wall time: 12.2 s
